In [1]:
import pandas as pd
import numpy as np

import pyomo.environ as pyo

import gurobipy as gp
from gurobipy import GRB

import openpyxl

from icecream import ic


In [2]:
file_path = "Railway services-2024.xlsx"

try:
    df = pd.read_excel(io=file_path, engine="openpyxl")
    ic(df.head())

except FileNotFoundError:
    print(f"File '{file_path}' not found. Please check the file path.")
    

ic| df.head():    Trip Departure Time Arrival Time From To  Demand(μ)  Demand(σ)  Line
               0     1       07:00:00     07:46:00    M  A        327         64   100
               1     2       07:00:00     07:57:00    H  M        936        308   800
               2     3       07:02:00     07:22:00    M  B        461        252   200
               3     4       07:04:00     08:03:00    M  J        428         10  1000
               4     5       07:06:00     08:13:00    M  F        449        124   600


In [3]:
# Data cleaning/ enhancing

# Removing unneccesary spaces from 'From' and 'To' columns. 
df['From'] = df['From'].str.strip()
df['To'] = df['To'].str.strip()

# Adding an indicator when the trip is located on line 400
indicator_line_400 = np.where(df['Line'] == 400.0, 1, 0)
df["line_400"] = indicator_line_400

ic(df)

ic| df:      Trip Departure Time Arrival Time From To  Demand(μ)  Demand(σ)  Line  \
        0       1       07:00:00     07:46:00    M  A        327         64   100   
        1       2       07:00:00     07:57:00    H  M        936        308   800   
        2       3       07:02:00     07:22:00    M  B        461        252   200   
        3       4       07:04:00     08:03:00    M  J        428         10  1000   
        4       5       07:06:00     08:13:00    M  F        449        124   600   
        ..    ...            ...          ...  ... ..        ...        ...   ...   
        195   196       08:57:00     09:30:00    D  M        384        314   400   
        196   197       08:58:00     09:26:00    M  D        395        366   400   
        197   198       08:58:00     09:33:00    M  E        331        436   500   
        198   199       08:59:00     10:02:00    F  M        795         12   600   
        199   200       08:59:00     09:45:00    K  M       1034 

,Trip,Departure Time,Arrival Time,From,To,Demand(μ),Demand(σ),Line,line_400
0,1,07:00:00,07:46:00,M,A,327,64,100,0
1,2,07:00:00,07:57:00,H,M,936,308,800,0
2,3,07:02:00,07:22:00,M,B,461,252,200,0
3,4,07:04:00,08:03:00,M,J,428,10,1000,0
4,5,07:06:00,08:13:00,M,F,449,124,600,0
...,...,...,...,...,...,...,...,...,...
195,196,08:57:00,09:30:00,D,M,384,314,400,1
196,197,08:58:00,09:26:00,M,D,395,366,400,1
197,198,08:58:00,09:33:00,M,E,331,436,500,0
198,199,08:59:00,10:02:00,F,M,795,12,600,0


In [4]:
### CREATING THE MODEL

# Creation of the Model:
model = pyo.ConcreteModel()

# Creation of sets:

train_type = [
    "OC", 
    "OH"
    ]
cost_train_type = {
    "OC": 260,
    "OH": 210
}
length_train_type = {
    "OC": 100, 
    "OH": 70
    }
capacity_train_type = {
    "OC": 620, 
    "OH": 420
    }
cross_section = set(
    df['From'].values + df['To'].values
    )

# Cross section based
cross_sections_using_line400 = set(
    df.loc[df['line_400'] == 1, 'From'].values + df.loc[df['line_400'] == 1, 'To'].values
)
# OR Trip based (Comment out for usage of section based version)
cross_sections_using_line400 = (df.loc[df['line_400'] == 1]).index.to_list()

# Regular sets
model.trips = pyo.Set(
    initialize = df['Trip']
    )
#model.trips.pprint()

model.train_type = pyo.Set(
    initialize = train_type
    )
#model.train_type.pprint()

#FIXME: Returns the warning about unordered data. 
#model.cross_section = pyo.Set(
#    initialize = cross_section
#    )
#model.cross_section.pprint()

# Index sets
model.index_set_allocation = pyo.Set(
    initialize = model.trips * model.train_type
    )
#model.index_set_allocation.pprint()

# Creating the Parameters. 
model.cost_train_type = pyo.Param(
    model.train_type, 
    initialize = cost_train_type
)
#model.cost_train_type.pprint()

model.length_train_type = pyo.Param(
    model.train_type, 
    initialize = length_train_type
    )
#model.length_train_type.pprint()

model.capacity_train_type = pyo.Param(
    model.train_type, 
    initialize = capacity_train_type
    )
#model.capacity_train_type.pprint()

model.trips_using_line400 = pyo.Param(
    model.trips,  # Assuming model.cross_section contains all possible cross sections
    initialize={trip: 1 if trip in cross_sections_using_line400 else 0 for trip in model.trips}
)
#model.trips_using_line400.pprint()

model.passengers_per_trip = pyo.Param(
    model.trips, 
    initialize = {trip: df['Demand(μ)'].loc[trip - 1] for trip in model.trips} # -1 because the trips start with trip 1, but the df with index 0. So, loc[200] results in an error, because there are only 199 records. 
)
#model.passengers_per_trip.pprint()

# Creation of the Variables
model.allocation_train_numbers = pyo.Var(
    model.index_set_allocation, 
    domain = pyo.NonNegativeIntegers, 
    name = 'train_allocation', 
    doc = 'The number of trains of a certain type allocated to a cross-section'
    )
#model.allocation_train_numbers.pprint()

#model.allocation_train_numbers.pprint()
#model.passengers_per_trip.pprint()


In [5]:
# Creation of constraints

# Rule for the combined train length
def rule_maximum_length(
        m, 
        trip
        ):
    combined_train_length = sum(
        m.length_train_type[(train_type)] * m.allocation_train_numbers[(trip, train_type)]
        for train_type in m.train_type
    )
    return combined_train_length <= 300 - 100 * m.trips_using_line400[(trip)]
model.constr_maximum_length = pyo.Constraint(
    model.trips, 
    rule = rule_maximum_length
    )
#model.constr_maximum_length.pprint()

def rule_difference_between_number_of_train_types1(m):
    return 0.75 * sum(
            m.allocation_train_numbers[(trip, train_type)] 
            for trip in m.trips 
            for train_type in m.train_type if train_type == "OC"
        ) <= sum(
            m.allocation_train_numbers[(trip, train_type)] 
            for trip in m.trips 
            for train_type in m.train_type if train_type == "OH"
        )
model.constr_difference_between_number_of_train_types1 = pyo.Constraint(
    rule=rule_difference_between_number_of_train_types1
)
#model.constr_difference_between_number_of_train_types1.pprint()

def rule_difference_between_number_of_train_types2(m):
    return 0.75 * sum(
            m.allocation_train_numbers[(trip, train_type)] 
            for trip in m.trips 
            for train_type in m.train_type if train_type == "OH"
        ) <= sum(
            m.allocation_train_numbers[(trip, train_type)] 
            for trip in m.trips 
            for train_type in m.train_type if train_type == "OC"
        )
model.constr_difference_between_number_of_train_types2 = pyo.Constraint(
    rule=rule_difference_between_number_of_train_types2
)
#model.constr_difference_between_number_of_train_types2.pprint()


def obj_minimize_total_cost(m):
    total_cost = sum(
        m.cost_train_type[(train_type)] * sum(
            m.allocation_train_numbers[(trip, train_type)] 
            for trip in m.trips
        )
        for train_type in m.train_type
    )
    return total_cost
model.objective = pyo.Objective(
    rule=obj_minimize_total_cost,
    sense=pyo.minimize
)
#model.objective.pprint()


In [6]:
# Rule for the required combination of the number of trains per train type based on the expected number of passengers. 
def rule_passenger_limit(
        m, 
        trip
        ):
    available_capacity = sum(
        m.capacity_train_type[(train_type)] * m.allocation_train_numbers[(trip, train_type)]
        for train_type in m.train_type
    )
    return available_capacity >= int(m.passengers_per_trip[(trip)])
model.constr_passenger_limit = pyo.Constraint(
    model.trips, 
    rule = rule_passenger_limit
    )
#model.constr_passenger_limit.pprint()


def extra1(m, trip):
    return m.allocation_train_numbers[(trip, "OH")] >= 1
def extra2(m, trip):
    return m.allocation_train_numbers[(trip, "OC")] >= 1

#model.extra1 = pyo.Constraint(model.trips, rule=extra1)
#model.extra2 = pyo.Constraint(model.trips, rule=extra2)



In [7]:
# Solving the problem.

solver = pyo.SolverFactory('cbc')

# Solve the model
results = solver.solve(model, tee=True, options={'seconds': 60})

#FIXME: The passenger limit and length limit contradict each other for some reason. That is why it is infeasable. 

# Check the solver status
if (results.solver.status == pyo.SolverStatus.ok) and (results.solver.termination_condition == pyo.TerminationCondition.optimal):
    print("Solver terminated successfully. Model is feasible.")
elif results.solver.termination_condition == pyo.TerminationCondition.infeasible:
    print("Solver terminated: Model is infeasible.")
else:
    print("Solver terminated with non-optimal solution.")


ic(results)

solution_dict = {}

for trip in model.trips:
    for train_type in model.train_type:
        solution_dict[(trip, train_type)] = model.allocation_train_numbers[(trip, train_type)].value

solution_DF = pd.DataFrame(index=model.trips, columns=model.train_type)

for trip in model.trips:
    for train_type in model.train_type:
        solution_DF.loc[trip, train_type] = solution_dict.get((trip, train_type), np.nan)

ic(results.solver.time)
#ic(solution_dict)
ic(solution_DF)


Welcome to the CBC MILP Solver 
Version: 2.10.10 
Build Date: Jul 24 2023 

command line - /opt/homebrew/bin/cbc -seconds 60 -printingOptions all -import /var/folders/sd/gjs1t8tx7hg3t1h6wmfr_9k00000gn/T/tmpunlmb6l3.pyomo.lp -stat=1 -solve -solu /var/folders/sd/gjs1t8tx7hg3t1h6wmfr_9k00000gn/T/tmpunlmb6l3.pyomo.soln (default strategy 1)
seconds was changed from 1e+100 to 60
Option for printingOptions changed from normal to all
Presolve 402 (0) rows, 400 (0) columns and 1600 (0) elements
Statistics for presolved model
Original problem has 400 integers (0 of which binary)
==== 0 zero objective 2 different
200 variables have objective of 210
200 variables have objective of 260
==== absolute objective values 2 different
200 variables have objective of 210
200 variables have objective of 260
==== for integers 0 zero objective 2 different
200 variables have objective of 210
200 variables have objective of 260
==== for integers absolute objective values 2 different
200 variables have objective

ic| results: {'Problem': [{'Name': 'unknown', 'Lower bound': None, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 402, 'Number of variables': 400, 'Number of binary variables': 0, 'Number of integer variables': 400, 'Number of nonzeros': 400, 'Sense': 'minimize'}], 'Solver': [{'Status': 'warning', 'User time': -1.0, 'System time': 0.0, 'Wallclock time': 0.0, 'Termination condition': 'infeasible', 'Termination message': 'Model was proven to be infeasible.', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': None, 'Number of created subproblems': None}}, 'Error rc': 0, 'Time': 0.015882015228271484}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}
ic| results.solver.time: 0.015882015228271484
ic| solution_DF:        OC    OH
                 1    None  None
                 2    None  None
                 3    None  None
                 4    None  None
                 5    None  None
            

Solver terminated: Model is infeasible.


,OC,OH
1,None,None
2,None,None
3,None,None
4,None,None
5,None,None
...,...,...
196,None,None
197,None,None
198,None,None
199,None,None


In [8]:
from itertools import product

# Define the lengths of each train type
OH_length = 100
OC_length = 70

OH_cap = 420
OC_cap = 620

# Define the maximum length of a train
max_train_length = 300

# Generate all possible combinations of OH and OC trains
all_combinations = {}

for num_OH in range(max_train_length // OH_length + 1):
    for num_OC in range(max_train_length // OC_length + 1):

        length = num_OH * OH_length + num_OC * OC_length
        if length <= max_train_length:
            combination = ()
            i = 0
            while i <= num_OH:
                if not i == 0:
                    combination += ('OH',)
                i += 1
            
            i = 0
            while i <= num_OC:
                if not i == 0:
                    combination += ('OC',)
                i += 1

            cap = num_OH * OH_cap + num_OC * OC_cap
            all_combinations[combination] = (length, cap)

ic(all_combinations)


ic| all_combinations: {(): (0, 0),
                       ('OC',): (70, 620),
                       ('OC', 'OC'): (140, 1240),
                       ('OC', 'OC', 'OC'): (210, 1860),
                       ('OC', 'OC', 'OC', 'OC'): (280, 2480),
                       ('OH',): (100, 420),
                       ('OH', 'OC'): (170, 1040),
                       ('OH', 'OC', 'OC'): (240, 1660),
                       ('OH', 'OH'): (200, 840),
                       ('OH', 'OH', 'OC'): (270, 1460),
                       ('OH', 'OH', 'OH'): (300, 1260)}


{(): (0, 0),
 ('OC',): (70, 620),
 ('OC', 'OC'): (140, 1240),
 ('OC', 'OC', 'OC'): (210, 1860),
 ('OC', 'OC', 'OC', 'OC'): (280, 2480),
 ('OH',): (100, 420),
 ('OH', 'OC'): (170, 1040),
 ('OH', 'OC', 'OC'): (240, 1660),
 ('OH', 'OH'): (200, 840),
 ('OH', 'OH', 'OC'): (270, 1460),
 ('OH', 'OH', 'OH'): (300, 1260)}